Act as a generative AI expert who have lots of experience in working with Langchain RAG, now I have a reasearch paper like pdf, now I know how to perform rag on a simple pdf where the content is written in normal way, but its difficult to do it in pdf's where content is divided into 2 section in vertical way, as you can see in the attached image now in this page of the pdf rag loader and splitter can easily correctly fetch the data from blue annotated area, but when its come to red annotated area you can see its 2 different section with 2 different content, when when rag is going to read it its going to read it left to right 1 line at a time so in this case it is going to mix both different content's first line in one. so My question is how I am going to load and split the data of this pdf in a way that can appropriatly identify first where there is 2 section content(like Red annotated area) and 1 sedition content (like Blue annotated area), and then how its going to read the text from each section seperatly in red annotated area that does not mix the content of first red annotated area content with second red annotated area content. please provide some solution.   

what is there is graphs and tables in the single or two column layout how it will be managed in txt file you ae saving it in, dont say multimodel rag, I know about multimodel rag that is for perform proper image identification and image embedding and store it in the vector db, I am not asking about that, what I am asking is with your current logic How you are going to identify if there if a image, if there is image weather its in single column layout or two column layout, if its in 2 column layout weather its part of left column content or right column content, and finally how you are going to save it and do the splitting with the appropriate section??

In [1]:
!pip install pdfplumber pymupdf langchain-openai langchain_community faiss-cpu openai langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/AdvanceRAG Problems

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/AdvanceRAG Problems


In [3]:
#"lm.php-2-8.pdf"

In [4]:
import os
import pdfplumber
import fitz  # PyMuPDF
import base64

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.chat_history import InMemoryChatMessageHistory # Corrected import path
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain.storage import InMemoryStore # Corrected import path
from openai import OpenAI

# Create folders to save images and tables if they do not exist
OUTPUT_DIR = "pdf_assets"
os.makedirs(f"{OUTPUT_DIR}/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/tables", exist_ok=True)

###Define the Document Layout Analysis function
This handles:
1. Detecting one vs two columns
2. Zone segmentation with bounding boxes
3. Reading order: left column first, right column second
4. Image extraction
5. Table extraction

###Document Layout Analysis (DLA)<br>

The general process of detecting and classifying structural components in a document (e.g., text blocks, columns, tables, images, headers, footers). Commonly done for multi-column scientific papers, newspapers, patents, etc.


In [5]:
#Document Layout Analysis (DLA)
def process_pdf_layout(file_path):
    """
    Process PDF pages for layout analysis, zone segmentation, reading order,
    and embedded object extraction.
    """
    text_chunks = []

    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            width = page.width
            height = page.height
            #Zone Segmentation / Region Segmentation
            # Heuristic: use width to guess if it is two-column layout
            is_two_column = width > 500

            if is_two_column:
                # Define bounding boxes for left and right columns
                left_bbox = (0, 0, width / 2, height)
                right_bbox = (width / 2, 0, width, height)

                # Reading Order Detection
                # Another key aspect: after zones are identified, you need to determine the correct sequence to read
                # them in (e.g., left column top-down, then right column top-down).

                # Extract left column text
                left_text = page.within_bbox(left_bbox).extract_text() or ""
                if left_text.strip():
                    text_chunks.append(left_text)

                # Extract right column text
                right_text = page.within_bbox(right_bbox).extract_text() or ""
                if right_text.strip():
                    text_chunks.append(right_text)

            else:
                # For single-column pages, extract entire page text
                full_text = page.extract_text() or ""
                if full_text.strip():
                    text_chunks.append(full_text)

    # Extract images using PyMuPDF
    doc = fitz.open(file_path)
    for page_num, page in enumerate(doc):
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            img_ext = base_image["ext"]
            image_filename = f"{OUTPUT_DIR}/images/page{page_num+1}_img{img_index+1}.{img_ext}"
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
            text_chunks.append(f"[[IMAGE: {image_filename}]]")

    # Extract tables using pdfplumber
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for table_index, table in enumerate(tables):
                rows = []
                for row in table:
                    cleaned_row = [cell if cell else "" for cell in row]
                    rows.append(" | ".join(cleaned_row))
                table_text = "\n".join(rows)
                table_filename = f"{OUTPUT_DIR}/tables/page{page_num+1}_table{table_index+1}.txt"
                with open(table_filename, "w") as f:
                    f.write(table_text)
                text_chunks.append(f"[[TABLE: {table_filename}]]")

    return text_chunks

In [6]:
#Putting it together for your pipeline
pdf_text_chunks = process_pdf_layout("lm.php-2-8.pdf")

print("Number of raw chunks:", len(pdf_text_chunks))
print("Example chunk:\n", pdf_text_chunks[0][:300])

Number of raw chunks: 66
Example chunk:
 Published in Journal of Optics A: Pure an
which should be used for
Comparing glass and
microlenses fabricate
technologies
HOttevaere1,RCox2,HPHerzig3,TMiyashita4,
andHThienpont1
1DepartmentofAppliedPhysicsandPhotonics(TONA-IR),VrijeUni
2MicroFabTechnologies,Inc.,1104SummitAvenue,Suite110,Plano
3Inst


In [42]:
pdf_text_chunks

['Published in Journal of Optics A: Pure an\nwhich should be used for\nComparing glass and\nmicrolenses fabricate\ntechnologies\nHOttevaere1,RCox2,HPHerzig3,TMiyashita4,\nandHThienpont1\n1DepartmentofAppliedPhysicsandPhotonics(TONA-IR),VrijeUni\n2MicroFabTechnologies,Inc.,1104SummitAvenue,Suite110,Plano\n3InstituteofMicrotechnology,UniversityofNeuchaˆtel,RueA-LBreg\n4RicohCompanyLtd,ResearchandDevelopmentGroup,3-2-3,Shin-\n5DepartmentofInformationTechnology,GhentUniversity—IMEC,S\n6Physics,SchoolofEngineeringandPhysicalSciences,Heriot–Watt\n7SUSSMicroOpticsSA,Jaquet-Droz7,CP42,CH-2007Neuchaˆtel,\n8KoreaInstituteofGeoscienceandMineralResources,POBox111,\nE-mail:hottevaere@tona.vub.ac.be\nAbstract\nWereviewthemostimportantfabricationtechniquesforg\nquantitativelycharacterizeinasystematicway thecorres\nfrom selectedresearchgroups. Forallourmeasurements\nprofiler,atransmissionMach–Zehnder interferometerand\nanddiscussthedifferentfabricationtechniquesbycompa\nmicrolenses,therangeofmaterials

In [7]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

split_chunks = []
for chunk in pdf_text_chunks:
    split_pieces = splitter.split_text(chunk)
    split_chunks.extend(split_pieces)

print("Number of split chunks:", len(split_chunks))

Number of split chunks: 87


In [8]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_')
client = OpenAI()

In [9]:
def describe_image(image_path):
    """
    Generates a text description for an image using OpenAI Vision.
    """
    with open(image_path, "rb") as img_file:
        img_b64 = base64.b64encode(img_file.read()).decode()

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe this image for an NLP research context."},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64}"}}
                ]
            }
        ]
    )
    return response.choices[0].message.content

In [10]:
documents = []

for chunk in split_chunks:
    if "[[IMAGE:" in chunk:
        image_path = chunk.split("[[IMAGE:")[1].split("]]")[0].strip()
        description = describe_image(image_path)
        chunk = chunk.replace(f"[[IMAGE: {image_path}]]", f"Image Description: {description}")

    if "[[TABLE:" in chunk:
        table_path = chunk.split("[[TABLE:")[1].split("]]")[0].strip()
        with open(table_path, "r") as f:
            table_content = f.read()
        chunk = chunk.replace(f"[[TABLE: {table_path}]]", f"Table:\n{table_content}")

    documents.append(Document(page_content=chunk))

print("Number of final documents:", len(documents))

Number of final documents: 87


In [11]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

print("FAISS vector store created.")

/tmp/ipython-input-11-3098457605.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS vector store created.


In [12]:
import polars as pl
import numpy as np

# Generate embeddings for the documents again (for demonstration)
document_contents = [doc.page_content for doc in documents]

# Let's create a simple DataFrame with the document content
df_docs = pl.DataFrame({
    "content": document_contents,
    "doc_index": range(len(document_contents))
})

print("\nPolars DataFrame created from document content:")
print(df_docs.head())


Polars DataFrame created from document content:
shape: (5, 2)
┌─────────────────────────────────┬───────────┐
│ content                         ┆ doc_index │
│ ---                             ┆ ---       │
│ str                             ┆ i64       │
╞═════════════════════════════════╪═══════════╡
│ Published in Journal of Optics… ┆ 0         │
│ microlenses,therangeofmaterial… ┆ 1         │
│ Applied Optics 8, issue 7, 407… ┆ 2         │
│ Makingthesephotonicsystemsfast… ┆ 3         │
│ DDrriivveerr TThheerrmmaallrre… ┆ 4         │
└─────────────────────────────────┴───────────┘


In [13]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"lambda_mult": 0.5}
)

print("Retriever with MMR ready.")

Retriever with MMR ready.


In [28]:
from langchain_core.runnables import RunnableLambda
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a helpful research assistant. "
        "Use the provided context to answer the question accurately."
    ),
    HumanMessagePromptTemplate.from_template(
        "Context: {context}\n\nQuestion: {question}"
    )
])

wrapped_retriever = RunnableLambda(lambda x: retriever.invoke(x["question"]))
llm = ChatGroq(
    model="llama3-70b-8192",
    api_key=userdata.get('GROQ_API_KEY')
)

rag_chain = (
    {"context": wrapped_retriever, "question": lambda x: x["question"]}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG chain is ready.")

RAG chain is ready.


In [29]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from uuid import uuid4

# Use a callable that returns a new or existing memory for a given session
session_id = str(uuid4())

# Keep the store in a dict if you want to reuse for multiple sessions
session_histories = {}

def get_message_history(session_id):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]

In [30]:
rag_chain_with_memory = RunnableWithMessageHistory(
    rag_chain,
    get_message_history,
    input_messages_key="question",
    history_messages_key="history"
)
print("InMemoryStore initialized for managing chat memory.")

InMemoryStore initialized for managing chat memory.


In [32]:
# First question
response1 = rag_chain_with_memory.invoke(
    {"question": "Explain Architecture of Planar GRIN microlenses?"},
    config={"configurable": {"session_id": session_id}}
)
print("\nAnswer 1:\n", response1)


Answer 1:
 Based on the provided documents, the architecture of Planar GRIN (Gradient Refractive Index) microlenses is not explicitly explained. However, from the information available, we can infer some details about the fabrication process and characteristics of Planar GRIN microlenses.

From Document(id='4487a985-cfa2-4b26-b11f-10f5d4f46266'), we can see that the fabrication process of refractive reflow photoresist microlenses involves the following steps:

1. Patterning of the photoresist layer (Figure 5(a))
2. Development of the exposed regions (Figure 5(b))
3. Melting of the cylindrical islands (Figure 5(c))

Additionally, the document mentions that the planar GRIN lenses have good imaging properties and are used in various applications such as photocopiers, facsimile systems, bar-code readers, and optical communication systems for light coupling purposes.

Unfortunately, the provided documents do not explicitly describe the architecture of Planar GRIN microlenses. To gather mor

In [34]:
# Follow-up question
response2 = rag_chain_with_memory.invoke(
    {"question": "Flow chart of a microlens fabrication process."},
    config={"configurable": {"session_id": session_id}}
)
print("\nAnswer 2:\n", response2)


Answer 2:
 The flow chart of a microlens fabrication process is shown in Figure 2 of Document(id='44173f3b-b076-42ec-ad68-d7cc3f985ade'). It displays the logical sequence of the different steps to be taken from the underlying physics.


In [37]:
# Follow-up question
response3 = rag_chain_with_memory.invoke(
    {"question": "Explain plastics are polycarbonate (PC) and poly(methyl methacrylate)(PMMA), and what is LIGA processing"},
    config={"configurable": {"session_id": session_id}}
)
print("\nAnswer 3:\n", response3)


Answer 3:
 Based on the provided context, it appears that polycarbonate (PC) and poly(methyl methacrylate) (PMMA) are two of the most commonly used plastics in optical components or in combination with power dissipating semiconductor structures.

However, there is no mention of LIGA processing in the provided context. LIGA is a German acronym that stands for Lithographie, Galvanoformung, Abformung, which translates to Lithography, Electroplating, and Molding. It is a fabrication process used to create microstructures, including microlenses. 

Although it's not mentioned in the provided context, LIGA processing is a well-known technique in the field of microlens fabrication. It involves the use of X-ray lithography to pattern a thick photoresist, followed by electroplating to create a metal mold, and finally, molding to replicate the microstructure.

It's possible that LIGA processing might be mentioned in other related documents or literature, but it's not present in the provided cont

In [38]:
# Follow-up question
response4 = rag_chain_with_memory.invoke(
    {"question": "Explain The different fabrication process steps for refractive reflow photoresist microlenses, and is it related to Thermal reflow or resist melting method"},
    config={"configurable": {"session_id": session_id}}
)
print("\nAnswer 4:\n", response4)


Answer 4:
 Based on the provided documents, the fabrication process steps for refractive reflow photoresist microlenses can be explained as follows:

1. The photoresist is heated above its glass transition temperature.
2. Due to surface tension, the shape of the photoresist cylinders changes to minimize the surface energy.
3. As a result, hemispherical microlenses are formed (see figure 5).

This technology has also been extended to yield arrays of cylindrical lenses and lenses with a square footprint.

The refractive reflow photoresist microlenses fabrication process is related to the Thermal reflow or resist melting method. In fact, the term "reflow" in this context refers to the process of heating the photoresist above its glass transition temperature, causing it to flow and change shape due to surface tension.

The Thermal reflow or resist melting method is a technique used to fabricate microlenses, where the photoresist is heated to a temperature above its glass transition point,

In [39]:
# Show that memory is populated
print("\n--- Memory turns stored ---\n")
for msg in session_histories[session_id].messages:
    print(f"{msg.type.upper()}: {msg.content}\n")


--- Memory turns stored ---

HUMAN: How does AxFormer perform accuracy-driven pruning for transformers?

AI: I apologize, but the provided context does not mention AxFormer or accuracy-driven pruning for transformers. The context appears to be related to photonics, microlens arrays, image description, molten salt, and glass matrix, but it does not provide any information about AxFormer or transformer pruning.

If you could provide more context or clarify the question, I'll do my best to assist you.

HUMAN: Explain Architecture of Planar GRIN microlenses?

AI: Based on the provided documents, the architecture of Planar GRIN (Gradient Refractive Index) microlenses is not explicitly explained. However, from the information available, we can infer some details about the fabrication process and characteristics of Planar GRIN microlenses.

From Document(id='4487a985-cfa2-4b26-b11f-10f5d4f46266'), we can see that the fabrication process of refractive reflow photoresist microlenses involves t

In [41]:
if session_id in session_histories:
  session_histories[session_id].messages.clear()
  print(f"Memory for session {session_id} has been cleared.")
else:
  print(f"No memory found for session {session_id}.")

# Verify that the memory is empty
print("\n--- Memory turns after clearing ---\n")
if session_id in session_histories:
    for msg in session_histories[session_id].messages:
        print(f"{msg.type.upper()}: {msg.content}\n")
else:
    print(f"Session {session_id} no longer in history store after clearing.")

Memory for session 63ddf5d3-d5ec-4caa-b8b7-c330f6167c77 has been cleared.

--- Memory turns after clearing ---



In [ ]:
def process_pdf_layout(file_path):
    """
    Process PDF pages with metadata:
    - Text chunks carry page number.
    - Images/tables carry page number and file path.
    """
    pagewise_chunks = []

    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            width = page.width
            height = page.height

            is_two_column = width > 500

            if is_two_column:
                left_bbox = (0, 0, width / 2, height)
                right_bbox = (width / 2, 0, width, height)

                left_text = page.within_bbox(left_bbox).extract_text() or ""
                if left_text.strip():
                    pagewise_chunks.append({
                        "type": "text",
                        "content": left_text,
                        "page_number": page_num
                    })

                right_text = page.within_bbox(right_bbox).extract_text() or ""
                if right_text.strip():
                    pagewise_chunks.append({
                        "type": "text",
                        "content": right_text,
                        "page_number": page_num
                    })

            else:
                full_text = page.extract_text() or ""
                if full_text.strip():
                    pagewise_chunks.append({
                        "type": "text",
                        "content": full_text,
                        "page_number": page_num
                    })

    doc = fitz.open(file_path)
    for page_num, page in enumerate(doc, start=1):
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            img_ext = base_image["ext"]
            image_filename = f"{OUTPUT_DIR}/images/page{page_num}_img{img_index+1}.{img_ext}"
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
            pagewise_chunks.append({
                "type": "image",
                "content": f"[[IMAGE: {image_filename}]]",
                "page_number": page_num,
                "file": image_filename
            })

    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables()
            for table_index, table in enumerate(tables):
                rows = []
                for row in table:
                    cleaned_row = [cell if cell else "" for cell in row]
                    rows.append(" | ".join(cleaned_row))
                table_text = "\n".join(rows)
                table_filename = f"{OUTPUT_DIR}/tables/page{page_num}_table{table_index+1}.txt"
                with open(table_filename, "w") as f:
                    f.write(table_text)
                pagewise_chunks.append({
                    "type": "table",
                    "content": f"[[TABLE: {table_filename}]]",
                    "page_number": page_num,
                    "file": table_filename
                })

    return pagewise_chunks

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

split_chunks = []

for chunk in pdf_text_chunks:
    if chunk["type"] == "text":
        splits = splitter.split_text(chunk["content"])
        for piece in splits:
            split_chunks.append({
                "type": "text",
                "content": piece,
                "page_number": chunk["page_number"]
            })
    else:
        # For image/table keep original
        split_chunks.append(chunk)

print("Number of split chunks:", len(split_chunks))

In [ ]:
documents = []

for chunk in split_chunks:
    content = chunk["content"]
    metadata = {"page_number": chunk["page_number"], "type": chunk["type"]}

    if chunk["type"] == "image":
        image_path = chunk["file"]
        description = describe_image(image_path)
        content = f"Image (page {chunk['page_number']}): {description}"

    elif chunk["type"] == "table":
        table_path = chunk["file"]
        with open(table_path, "r") as f:
            table_content = f.read()
        content = f"Table (page {chunk['page_number']}):\n{table_content}"

    documents.append(Document(page_content=content, metadata=metadata))

print("Number of final documents with metadata:", len(documents))